In [17]:
# ==============================================================================
# STEP 0: ENVIRONMENT SETUP & LIBRARIES INSTALLATION
# ==============================================================================
# First, we will install all the necessary libraries.
# This cell needs to be run only once.

print("Installing necessary libraries... Please wait.")
!pip install pyspark -q
!pip install rpy2 -q
!pip install plotly -q
!pip install xgboost -q
!pip install beautifulsoup4 -q
!pip install requests -q
print("All libraries installed successfully!")

# Import the libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import xgboost as xgb
import os

# Enable the R magic command for Colab
%load_ext rpy2.ipython

print("Setup complete. You can now proceed with the project.")

Installing necessary libraries... Please wait.
All libraries installed successfully!
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
Setup complete. You can now proceed with the project.


In [18]:
# ==============================================================================
# STEP 1: CONNECTING GOOGLE DRIVE TO COLAB
# ==============================================================================
# It's a best practice to keep our data in Google Drive.
# This ensures that the data remains safe even if the Colab session expires.

from google.colab import drive
drive.mount('/content/drive')

# IMPORTANT: Upload your 'sales_data.csv' file to your Google Drive
# and provide the correct path here. For example:
# '/content/drive/My Drive/Colab Notebooks/Project_MarketPulse/sales_data.csv'
file_path = '/content/drive/MyDrive/sales_data.csv' # <-- CHANGE YOUR PATH HERE

# Check if the file exists at the specified path
if os.path.exists(file_path):
    print(f"File found at: {file_path}")
else:
    print(f"ERROR: File not found at the specified path. Please check the path and upload the file.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ERROR: File not found at the specified path. Please check the path and upload the file.


In [20]:
# ==============================================================================
# STEP 2: LOADING AND UNDERSTANDING THE HISTORICAL SALES DATA
# ==============================================================================
# We will load our historical sales CSV file into a Pandas DataFrame.

df = pd.read_csv("/content/drive/MyDrive/indian_ecommerce_sales.csv")

print("Data loaded successfully. Here are the first 5 rows:")
print(df.head())

print("\nBasic information about the dataset:")
df.info()

print("\nStatistical summary of the dataset:")
print(df.describe())

Data loaded successfully. Here are the first 5 rows:
         Date  OrderID ProductID            ProductName        Category  \
0  2024-10-16  ORD1001    PID214     Ceramic Dinner Set  Home & Kitchen   
1  2023-04-25  ORD1002    PID241     Children Storybook           Books   
2  2023-01-26  ORD1003    PID208  Bamboo Chopping Board  Home & Kitchen   
3  2023-10-09  ORD1004    PID230           Pixel Tablet     Electronics   
4  2023-09-08  ORD1005    PID214     Ceramic Dinner Set  Home & Kitchen   

   UnitsSold  PricePerUnit_INR  DiscountPercentage   Region  
0          2          10669.55                  13  Kolkata  
1          4           1301.67                  32  Kolkata  
2          4           3882.84                  15  Chennai  
3         11          20124.94                  36    Delhi  
4          1          13851.46                   0  Chennai  

Basic information about the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns 

In [4]:
# ==============================================================================
# STEP 2: HISTORICAL SALES DATA LOAD KARNA AUR USKO SAMJHNA
# ==============================================================================
# Hum apni historical sales CSV file ko Pandas DataFrame mein load karenge.

df = pd.read_csv(file_path)

print("Data loaded successfully. Here are the first 5 rows:")
print(df.head())

print("\nDataset ki basic information:")
df.info()

print("\nDataset ki statistical summary:")
print(df.describe())

Data loaded successfully. Here are the first 5 rows:
         Date  OrderID ProductID            ProductName        Category  \
0  2024-10-16  ORD1001    PID214     Ceramic Dinner Set  Home & Kitchen   
1  2023-04-25  ORD1002    PID241     Children Storybook           Books   
2  2023-01-26  ORD1003    PID208  Bamboo Chopping Board  Home & Kitchen   
3  2023-10-09  ORD1004    PID230           Pixel Tablet     Electronics   
4  2023-09-08  ORD1005    PID214     Ceramic Dinner Set  Home & Kitchen   

   UnitsSold  PricePerUnit_INR  DiscountPercentage   Region  
0          2          10669.55                  13  Kolkata  
1          4           1301.67                  32  Kolkata  
2          4           3882.84                  15  Chennai  
3         11          20124.94                  36    Delhi  
4          1          13851.46                   0  Chennai  

Dataset ki basic information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 

In [21]:
# ==============================================================================
# STEP 3: DATA CLEANING AND PREPROCESSING
# ==============================================================================
# For a data analyst, this is the most critical step.

# 1. Checking for Null values
print("\nChecking for Null Values:")
print(df.isnull().sum())
# If there were any null values, we would fill or drop them, for example:
# df.dropna(inplace=True)

# 2. Correcting the data type of the 'Date' column (from object to datetime)
print("\nConverting 'Date' column to datetime format...")
df['Date'] = pd.to_datetime(df['Date'])

# 3. Creating New Features (Feature Engineering) - This makes the model smarter
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.day_name()
df['SaleAmount'] = df['UnitsSold'] * df['PricePerUnit_INR'] * (1 - df['DiscountPercentage']/100)

print("\nData after cleaning and feature engineering:")
print(df.head())


Checking for Null Values:
Date                  0
OrderID               0
ProductID             0
ProductName           0
Category              0
UnitsSold             0
PricePerUnit_INR      0
DiscountPercentage    0
Region                0
dtype: int64

Converting 'Date' column to datetime format...

Data after cleaning and feature engineering:
        Date  OrderID ProductID            ProductName        Category  \
0 2024-10-16  ORD1001    PID214     Ceramic Dinner Set  Home & Kitchen   
1 2023-04-25  ORD1002    PID241     Children Storybook           Books   
2 2023-01-26  ORD1003    PID208  Bamboo Chopping Board  Home & Kitchen   
3 2023-10-09  ORD1004    PID230           Pixel Tablet     Electronics   
4 2023-09-08  ORD1005    PID214     Ceramic Dinner Set  Home & Kitchen   

   UnitsSold  PricePerUnit_INR  DiscountPercentage   Region  Year  Month  Day  \
0          2          10669.55                  13  Kolkata  2024     10   16   
1          4           1301.67             

In [22]:
# ==============================================================================
# STEP 4: EXPLORATORY DATA ANALYSIS (EDA) - EXTRACTING INSIGHTS
# ==============================================================================
# Now, we will find patterns and trends from the data using interactive charts.

# 1. Monthly Sales Trend
monthly_sales = df.groupby(df['Date'].dt.to_period('M'))['SaleAmount'].sum().reset_index()
monthly_sales['Date'] = monthly_sales['Date'].dt.to_timestamp()
fig1 = px.line(monthly_sales, x='Date', y='SaleAmount', title='Monthly Sales Trend', labels={'Date':'Month', 'SaleAmount':'Total Sales (INR)'})
fig1.show()

# 2. Sales by Category
category_sales = df.groupby('Category')['SaleAmount'].sum().reset_index().sort_values(by='SaleAmount', ascending=False)
fig2 = px.bar(category_sales, x='Category', y='SaleAmount', title='Total Sales by Product Category', text_auto='.2s')
fig2.show()

# 3. Effect of Discount on Sales (Visual Analysis)
fig3 = px.scatter(df, x='DiscountPercentage', y='UnitsSold', title='Discount Percentage vs. Units Sold',
                  labels={'DiscountPercentage':'Discount (%)', 'UnitsSold':'Units Sold'},
                  trendline="ols", # This adds a regression trend line
                  color='Category')
fig3.show()
print("In the scatter plot above, you can see that as the discount increases, the trend for Units Sold also goes up.")

In the scatter plot above, you can see that as the discount increases, the trend for Units Sold also goes up.


In [26]:
# ==========================================================
# DEBUGGING CELL FOR STEP 5
# Is cell ko alag se chala kar check karein
# ==========================================================

# Pehle, yeh confirm karein ki aapki file sahi se load hui hai
# (Apna file path yahan sahi se daalein)
file_path = '/content/drive/MyDrive/indian_ecommerce_sales.csv' # <-- CHECK YOUR PATH HERE

try:
    # CSV file dobara load karke check karte hain
    df = pd.read_csv(file_path)
    df['DiscountPercentage'] = df['DiscountPercentage'].fillna(0) # Agar koi NaN value hai to use 0 kar de
    df['UnitsSold'] = df['UnitsSold'].fillna(0)

    print("CSV file successfully loaded for debugging.")

    # Ab data groups check karte hain
    discounted_sales = df[df['DiscountPercentage'] > 0]['UnitsSold']
    non_discounted_sales = df[df['DiscountPercentage'] == 0]['UnitsSold']

    print("\n--- DATA CHECK ---")
    print(f"Total sales entries WITH a discount: {len(discounted_sales)}")
    print(f"Total sales entries WITHOUT a discount: {len(non_discounted_sales)}")

    if len(discounted_sales) < 2 or len(non_discounted_sales) < 2:
        print("\n****************************************************************")
        print("PROBLEM MIL GAYA: T-Test chalane ke liye dono groups (discounted aur non-discounted) mein kam se kam 2 items hone chahiye.")
        print("Aapke data mein ek group khaali hai ya usmein sirf 1 item hai.")
        print("\nSOLUTION: Apni `sales_data.csv` file ko us 'Advanced Prompt' se dobara banayein jo maine diya tha, taaki usmein dono tarah ka data (discounted aur non-discounted) maujood ho.")
        print("****************************************************************")
    else:
        print("\nGOOD NEWS: Aapke data mein T-test ke liye paryaapt entries hain. Problem data mein nahi, kahin aur hai.")
        print("Please ab apna poora 'red color' wala error message zaroor bhejein.")

except FileNotFoundError:
    print("ERROR: File not found! Please check the file_path variable. Kya aapne Step 1 theek se chalaya tha?")
except Exception as e:
    print(f"An unexpected error occurred while checking the data: {e}")

CSV file successfully loaded for debugging.

--- DATA CHECK ---
Total sales entries WITH a discount: 416
Total sales entries WITHOUT a discount: 584

GOOD NEWS: Aapke data mein T-test ke liye paryaapt entries hain. Problem data mein nahi, kahin aur hai.
Please ab apna poora 'red color' wala error message zaroor bhejein.


In [24]:
# ==============================================================================
# STEP 6: MACHINE LEARNING - SALES FORECASTING MODEL (XGBoost)
# ==============================================================================
# Now, we will build a model to predict future sales.

# 1. Prepare the data for time-series forecasting (aggregate sales by day)
df_daily = df.set_index('Date').groupby(pd.Grouper(freq='D'))['SaleAmount'].sum().reset_index()
df_daily = df_daily.set_index('Date')
df_daily = df_daily[df_daily['SaleAmount'] > 0] # Remove days with zero sales

print("Daily aggregated sales data for forecasting:")
print(df_daily.head())

# 2. Create time-series features
def create_features(df):
    df = df.copy()
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    return df

df_daily_features = create_features(df_daily)

# 3. Define features (X) and target (y)
X = df_daily_features.drop('SaleAmount', axis=1)
y = df_daily_features['SaleAmount']

# 4. Split data into training and testing sets
# For time-series data, the split must be chronological, not random.
split_date = '2024-09-01'
X_train, X_test = X[:split_date], X[split_date:]
y_train, y_test = y[:split_date], y[split_date:]

# 5. Train the XGBoost model
model = xgb.XGBRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    early_stopping_rounds=50,
    objective='reg:squarederror'
)

print("\nTraining the XGBoost model...")
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=100) # This will print the evaluation metric every 100 rounds

# 6. Make predictions on the test set
y_pred = model.predict(X_test)

# 7. Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"\nModel Evaluation:")
print(f"Mean Absolute Error (MAE): {mae:,.2f} INR")
print(f"R-squared (R2 Score): {r2:.2f}")

# 8. Visualize the results
results_df = pd.DataFrame({'Actual Sales': y_test, 'Predicted Sales': y_pred})
fig = go.Figure()
fig.add_trace(go.Scatter(x=results_df.index, y=results_df['Actual Sales'], mode='lines', name='Actual Sales'))
fig.add_trace(go.Scatter(x=results_df.index, y=results_df['Predicted Sales'], mode='lines', name='Predicted Sales', line=dict(dash='dash')))
fig.update_layout(title='Actual vs. Predicted Sales Forecast', xaxis_title='Date', yaxis_title='Sales (INR)')
fig.show()

Daily aggregated sales data for forecasting:
             SaleAmount
Date                   
2023-01-01  319562.9160
2023-01-02   58063.7600
2023-01-03   11512.9371
2023-01-04  428475.2416
2023-01-06   71456.1400

Training the XGBoost model...
[0]	validation_0-rmse:167394.72109	validation_1-rmse:183599.56014
[93]	validation_0-rmse:134629.41375	validation_1-rmse:181148.13208

Model Evaluation:
Mean Absolute Error (MAE): 132,799.38 INR
R-squared (R2 Score): 0.07


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
